# Exploring data

## Working with dataframes  

Dataframes are a data structure specific to the pandas library in Python. Pandas gives us a wide range of functions for reading and writing data in a structured tabular format, and for cleaning, reshaping and viewing the data.

[Link to pandas documentation](https://pandas.pydata.org/docs/user_guide/10min.html)

### Fetching data

In [ ]:
# import the library
import pandas as pd # 'pd' is a common alias for pandas, to make it shorter to type when you need to call on it

In [ ]:
# declare a variable (df)
# read in the CSV file contents to be the value of the variable
df = pd.read_csv("employees.csv") # the file needs to be in the same directory as the Jupyter notebook

### Inspecting data

In [ ]:
# view the top n rows of the dataframe
# default n is 5
# change this by specifying inside the brackets e.g. df.head(2)
df.head()

In [ ]:
# view the dataframe's column headings
df.columns

In [ ]:
# see a random sample of a row from the dataframe
# default is one row
# change this by specifying inside the brackets e.g. df.sample(10)
df.sample()

In [ ]:
# view the shape of hte dataframe 
# (the number of rows and columns)
df.shape

In [ ]:
# view summary statistics for any numeric columns in the dataframe
df.describe()

### Filtering data

In [ ]:
# view or call on a single column from the dataframe
df['Start Date'] # if a column name has spaces, address it like this

In [ ]:
# filter rows depending on a specific condition
# e.g. the city name is 'dallas'
df_dallas = df.loc[df.City=="dallas"] # returns nothing! why?
df_dallas

In [ ]:
# filter rows based on position
df_mid = df.iloc[3:7]
df_mid

In [ ]:
# filter rows based on a comparison operator
df_retirement = df[df.Age > 58]
df_retirement

In [ ]:
# filter rows based on multiple conditions and a logical operator
# e.g. 'Age' is more than 58 AND 'State' is Florida 
df_age_state = df[(df.Age > 58) & (df.State == " Florida ")]
df_age_state

In [ ]:
# filter rows using a list of values to include
boolean_mask = df.City.isin([' dallas ', ' miami '])
df_list = df[boolean_mask]
df_list

## Cleaning and reshaping data

### Removing unnecessary data

In [ ]:
# delete unneeded columns 
df_no_age = df.drop(columns=['Age'])
df_no_age.head()

In [ ]:
# convert date values to datetime dtype
# if we don't do this, the values will not behave like dates
# and we can't use them for time series analysis
df['DateOfBirth'] = pd.to_datetime(df['DateOfBirth'])

# display datetime values in a preferred format
# note this does not change the underlying datetime value
df['DateOfBirth'] = df['DateOfBirth'].dt.strftime('%d/%m/%y')
df.head()

### NULL and NaN values  

  Some values might be blank in your data. Excel and Power BI can work with them, but pandas does not.  
  There are several built-in pandas functions to deal with these so they don't interfere with your analysis later.

In [ ]:
# remove rows with NULL values (be sure this is what you want to happen!)
#df = df.dropna(how='all') # drop the row if ALL the values are NULL or NaN
#df = df.dropna(how='any') # drop the row if ANY of the values are NULL or NaN

# OR fill any NULL or NaN values with a new value
df = df.fillna(0) # if the values are meant to be numeric, the number zero is a common filler

### Removing duplicate rows

In [ ]:
# identifies duplicate rows in the dataframe, based on all columns
df_dupes = df[df.duplicated()] 
# identifies duplicate rows in the dataframe, based on a subset of columns
df_dupe_name_age = df[df.duplicated(subset = ['Name', 'Age'])] 
df.drop_duplicates(inplace=True, keep='first')
# keep = 'first' keeps the first observed row, marks later ones as duplicates
# keep = last keeps the last observed duplicate row
# keep = False marks all duplicates

### Converting data types  
When we start looking at machine learning models, there will be times that we want categorical variables encoded as numbers. This is because many models will not treat a variable as a factor in the model if it is not numeric.

In [ ]:
# converting data types
# you can check data types for all columns using dtypes
print(df.dtypes)

In [ ]:
# converting categorical variables to integer values 1 or 0 (TRUE or FALSE)
# the option you specify in the conditions will be coded to '1' or TRUE
# everytihng else wil lbe coded to '0' or FALSE 
df['Gender'] = (df['Gender']==' female ').astype('int')

Before we encode categorical variables we should make sure the categories are clean and well defined. We may need to change cases, remove trailing and leading whitespace, and take out currency symbols.

In [ ]:
# tidying and manipulating text strings
# convert to upper case
df['Name'] = df['Name'].str.upper()
# instad of 'upper' try 'lower' (convert to lower case) and 'strip' (removes whitespace)
df.head()

In [ ]:
# replace substrings (parts of the text value)
# e.g. remove the currecny symbol from values to make them numeric
#df['Salary'] = df['Salary'].str.replace('$', '').astype('int')

In [ ]:
# extract part of the string
# index is inclusive of start, exclusive of stop
df['substring'] = df['Name'].str.slice(start=2, stop=9)
df.head()

In [ ]:
# search a string
# get the rows where the name contains 'alice'
contains_pattern = df['Name'].str.contains('austin')
filtered_data = df[contains_pattern]
print(filtered_data)

### Change scale or scope

#### Custom functions

In [ ]:
# applying a custom function to a series or to the entire dataframe
# for a series: each element of the original column is passed to the function
# for a dataframe: based on the axis (1 = row, 0 = column) 
# the entire row or column is passed to the function

# define a function to calculate age in months
def age_in_months(x):
    if isinstance(x, (float, int)):
        return x * 12
    return x

# apply the function to the Age column
df['AgeInMonths'] = df['Age'].apply(age_in_months)
df.head()

In [ ]:
# define a function to map genders to numerical values
mapping_dict = {'Female':1, 'Male': 0}

# apply the mapping to the Gender column
df['gender_numeric'] = df['Gender'].map(mapping_dict)
df.head()

In [ ]:
# define a value to replace 'None' values with 'Unknown'
def replace_none(x):
    if x is None:
        return 'Unknown'
    return x

# apply the function to the whole dataframe
df = df.applymap(replace_none) # older versions of Pandas
# df = df.map(replace_none) # newer versions of Pandas

df.head()

### Changing shape - GROUPBY

In [ ]:
# group the dataframe by column 'State' and calculate the sum of 'Salary'
grouped = df.groupby('State')['Salary'].sum()
# try some other aggregate functions - mean, median, mode
grouped

# Visualising data with pandas, matplotlib, and seaborn

## matplotlib

In [ ]:
# uncomment to install on first run
#%pip install matplotlib

import matplotlib.pyplot as plt

# Data representing monthly website traffic (in thousands)
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun']
traffic = [150, 200, 180, 220, 250, 210]
plt.plot()
# Create a line plot
plt.plot(months, traffic)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Sample data representing monthly website traffic (in thousands)
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun']
traffic = [150, 200, 180, 220, 250, 210]

# Create a line plot
plt.plot(months, traffic)

# Add labels and a title
plt.xlabel('Month')
plt.ylabel('Monthly Traffic (in Thousands)')
plt.title('Monthly Website Traffic')

plt.show()

In [ ]:
# Create a line plot with custom appearance
plt.plot(months, traffic, marker='o', linestyle='--', color='g')

# Add labels and a title
plt.xlabel('Month')
plt.ylabel('Monthly Traffic (in Thousands)')
plt.title('Monthly Website Traffic')

# Add grid to the plot
plt.grid(True)
# Display the plot
plt.show()

In [ ]:
# Sample data for two products' monthly revenue (in thousands of dollars)
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun']
product_a_revenue = [45, 55, 60, 70, 80, 75]
product_b_revenue = [35, 40, 50, 55, 70, 68]

# Create a line plot for Product A with a blue line and circular markers
plt.plot(months, product_a_revenue, marker='o', linestyle='-', color='blue', label='Product A')

# Create a line plot for Product B with a red dashed line and square markers
plt.plot(months, product_b_revenue, marker='s', linestyle='--', color='red', label='Product B')

# Add labels and a title
plt.xlabel('Month')
plt.ylabel('Monthly Revenue (in $1000)')
plt.title('Monthly Revenue Comparison')

# Display a legend to differentiate between Product A and Product B
plt.legend()

# Display the plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Expense categories
categories = ['Housing', 'Transportation', 'Food', 'Entertainment', 'Utilities']

# Monthly expenses for Alice, Bob, and Carol
alice_expenses = [1200, 300, 400, 200, 150]
bob_expenses = [1100, 320, 380, 180, 140]
carol_expenses = [1300, 280, 420, 220, 160]

# Create an array for the x-axis positions
x = list(range(len(categories)))

# Width of the bars, we need it as we intend to plot multiple bars.
bar_width = 0.2

# Create bars for Alice's expenses, Subract bar width from x array,
# such that it will be placed to the left.
plt.bar([i - bar_width for i in x], alice_expenses, width=bar_width, label='Alice', color='skyblue')

# Create bars for Bob's expenses
plt.bar(x, bob_expenses, width=bar_width, label='Bob', color='lightcoral')

# Create bars for Carol's expenses, Add bar width to x array,
# such that it will be placed to the right.
plt.bar([i + bar_width for i in x], carol_expenses, width=bar_width, label='Carol', color='lightgreen')

# Add labels, a title, and a legend
plt.xlabel('Expense Categories')
plt.ylabel('Monthly Expenses (USD)')
plt.title('Monthly Expenses Comparison')

# To show the category names at x-axis positions.
plt.xticks(x, categories)
plt.legend()

# Display the plot
plt.show()

In [ ]:
# A sample data for stores
# Store Size Represents the size of each store in 100sq.ft
stores = ['Store A', 'Store B', 'Store C', 'Store D', 'Store E']
customers = [120, 90, 150, 80, 200]
revenue = [20000, 18000, 25000, 17000, 30000]
store_size = [10, 5, 15, 8, 20]  

# Here we are scaling the store sizes for point sizes in the scatter plot
point_sizes = [size * 100 for size in store_size]

# To Create a scatter plot with different point sizes
# Here alpha controls the intensity of the color
plt.scatter(customers, revenue, s=point_sizes, c='skyblue', alpha=0.7, edgecolors='b')

# Add labels, a title, and a legend
plt.xlabel('Number of Customers')
plt.ylabel('Revenue (USD)')
plt.title('Relationship between Customers, Revenue, and Store Size')

# To Display the plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Sample exam scores data
exam_scores = [68, 72, 75, 80, 82, 84, 86, 90, 92, 95, 98, 100]

# Custom bin ranges
bin_ranges = [60, 70, 80, 90, 100]

# Create a histogram with custom bin ranges by assigning it to bins
plt.hist(exam_scores, bins=bin_ranges, color='lightblue', edgecolor='black', alpha=0.7)

# Add labels and a title
plt.xlabel('Exam Scores')
plt.ylabel('Frequency')
plt.title('Exam Scores Histogram with Custom Bins')

# Calculate and add a median line
sorted_scores = sorted(exam_scores)
n = len(sorted_scores)
if n % 2 == 0:
    median_score = (sorted_scores[n//2-1] + sorted_scores[n//2]) / 2
else:
    median_score = sorted_scores[n//2]
plt.axvline(median_score, color='red', linestyle='dashed', linewidth=2, label=f'Median Score: {median_score}')

# Add a legend
plt.legend()

# Display the plot
plt.show()

In [ ]:
# Product categories
categories = ['Electronics', 'Clothing', 'Home Decor', 'Books', 'Toys']

# Sales data for each category
sales = [3500, 2800, 2000, 1500, 1200]

# Explode a specific segment (e.g., 'Clothing')
# second value (0.1) is the amount by which the segment 'Clothing'
explode = (0, 0.1, 0, 0, 0)  

# Create a pie chart with explode and shadow
plt.pie(sales, labels=categories, explode=explode, shadow=True, autopct='%1.1f%%')
plt.title('Sales by Product Category')

# Display the plot
plt.show()

In [ ]:
import random

# Generate random data with outliers
random.seed(42)
data = [random.normalvariate(0, 1) for _ in range(100)] + [random.normalvariate(6, 1) for _ in range(10)]

# Create a box plot with outliers
plt.figure(figsize=(8, 6))  # Set the figure size
plt.boxplot(data, vert=False, patch_artist=True, 
            boxprops={'facecolor': 'lightblue'}, 
            flierprops={'marker': 'o', 'markerfacecolor': 'red', 
                        'markeredgecolor': 'red'})

# Add labels and a title
plt.xlabel('Values')
plt.title('Box Plot with Outliers')

# Display the plot
plt.grid(True)  # Add a grid for better readability
plt.show()

In [ ]:
# Create a sample correlation matrix
correlation_matrix = [[1.0, 0.8, 0.3, -0.2],
                      [0.8, 1.0, 0.5, 0.1],
                      [0.3, 0.5, 1.0, -0.4],
                      [-0.2, 0.1, -0.4, 1.0]]

# Create a heatmap for the correlation matrix
plt.imshow(correlation_matrix, cmap='coolwarm', vmin=-1, vmax=1, aspect='auto', origin='upper')

# Add a colorbar with the following commands
cbar = plt.colorbar()
cbar.set_label('Correlation', rotation=270, labelpad=20)

# Add labels and a title
plt.title('Correlation Matrix Heatmap')
plt.xticks(range(len(correlation_matrix)), ['Var1', 'Var2', 'Var3', 'Var4'])
plt.yticks(range(len(correlation_matrix)), ['Var1', 'Var2', 'Var3', 'Var4'])

plt.show()

In [ ]:
# Sample data for stack plot
quarters = ['Q1', 'Q2', 'Q3', 'Q4']
electronics = [10000, 12000, 11000, 10500]
clothing = [5000, 6000, 7500, 8000]
home_appliances = [7000, 7500, 8200, 9000]

# Create a stack plot
plt.figure(figsize=(10, 6))  # Set the figure size
plt.stackplot(quarters, electronics, clothing, home_appliances, labels=['Electronics', 'Clothing', 'Home Appliances'],
              colors=['blue', 'green', 'red'], alpha=0.7)

# Add labels, legend, and title
plt.xlabel('Quarters')
plt.ylabel('Sales ($)')
plt.title('Product Category Sales Over Quarters')
plt.legend(loc='upper left')

# Display the plot
plt.grid(True)
plt.show()

## seaborn

In [ ]:
# uncomment to install on first run
#%pip install seaborn

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# check available datasets in seaborn
sns.get_dataset_names()

In [ ]:
# Restaurant tips data 
# Downloaded from seaborn
tips = sns.load_dataset('tips')

tips.head()

## categorical plots

In [ ]:
# Count Plot
plt.figure(figsize=(8, 5))
sns.countplot(x="day", hue="day", data=tips, palette="Set3", legend=False)
plt.title("Count of Tips by Day of the Week")
plt.show()

In [ ]:
# Swarm Plot
plt.figure(figsize=(8, 5))
sns.swarmplot(x="day", hue="day",y="total_bill", data=tips, palette="viridis", legend=False)
plt.title("Total Bill Distribution by Day of the Week")
plt.xlabel("Day of the Week")
plt.ylabel("Total Bill ($)")
plt.show()

In [ ]:
# Point Plot
plt.figure(figsize=(8, 5))
sns.pointplot(x="day", hue="day", y="total_bill", data=tips, errorbar="sd", palette="pastel", legend=False)
plt.title("Average Total Bill by Day of the Week")
plt.xlabel("Day of the Week")
plt.ylabel("Average Total Bill ($)")
plt.show()

In [ ]:
# Categorical Box Plot
plt.figure(figsize=(8, 5))
sns.boxplot(x="time", hue="time", y="total_bill", data=tips, palette="coolwarm", legend=False)
plt.title("Total Bill Distribution by Meal Time")
plt.xlabel("Meal Time")
plt.ylabel("Total Bill ($)")
plt.show()

In [ ]:
# Categorical Violin Plot
plt.figure(figsize=(10, 8))
sns.violinplot(x="day", hue="day", y="total_bill", data=tips, palette="Set2", legend=False)
plt.title("Total Bill Distribution by Day of the Week")
plt.xlabel("Day of the Week")
plt.ylabel("Total Bill ($)")
plt.show()

In [ ]:
# Box Plot using cat plot
plt.figure(figsize=(8, 5))
sns.catplot(x="day", hue="day", y="total_bill", data=tips, kind="box", palette="coolwarm", legend=False)
plt.title("Total Bill Distribution by Day of the Week")
plt.xlabel("Day of the Week")
plt.ylabel("Total Bill ($)")
plt.show()

In [ ]:
# Violin Plot using cat plot
plt.figure(figsize=(8, 5))
sns.catplot(x="time", hue="time", y="total_bill", data=tips, kind="violin", palette="Set2", legend=False)
plt.title("Total Bill Distribution by Time of Day")
plt.xlabel("Time of Day")
plt.ylabel("Total Bill ($)")
plt.show()

## univariate plots

In [ ]:
# Rug Plot
plt.figure(figsize=(8, 5))
sns.rugplot(x=tips['total_bill'], height=0.5)
plt.title("Rug Plot of Total Bill")
plt.xlabel("Total Bill ($)")
plt.ylabel("Density")
plt.show()

In [ ]:
# Strip Plot for Univariate Visualization
plt.figure(figsize=(8, 5))
sns.histplot(data=tips['total_bill'], kde=True, edgecolor="none")
plt.title("Total Bill Distribution")
plt.xlabel("Total Bill ($)")
plt.show()

In [ ]:
# Box Plot for Univariate Visualization
plt.figure(figsize=(8, 5))
sns.boxplot(x=tips['total_bill'], color="#8ebad9")
plt.title("Total Bill Distribution")
plt.xlabel("Total Bill ($)")
plt.show()

In [ ]:
# Strip Plot for Univariate Visualization
plt.figure(figsize=(8, 5))
sns.stripplot(x=tips['total_bill'], color="#8ebad9", jitter=True)
plt.title("Total Bill Distribution")
plt.xlabel("Total Bill ($)")
plt.show()

## bivariate

In [ ]:
# Regression Plot
plt.figure(figsize=(8, 5))
sns.regplot(x="total_bill", y="tip", data=tips, scatter_kws={"color": "blue"}, line_kws={"color": "red"})
plt.title("Regression Plot of Total Bill vs. Tip")
plt.xlabel("Total Bill ($)")
plt.ylabel("Tip ($)")
plt.show()

In [ ]:
# Joint Plot
sns.jointplot(x="total_bill", y="tip", data=tips, kind="scatter")
plt.show()

In [ ]:
# Hexbin Plot
plt.figure(figsize=(8, 5))
sns.jointplot(x="total_bill", y="tip",kind='hex', data=tips, gridsize=15, cmap="Blues")
plt.title("Hexbin Plot of Total Bill vs. Tip")
plt.xlabel("Total Bill ($)")
plt.ylabel("Tip ($)")
plt.show()

## multivariate plots

In [ ]:
# Violin Plot with Hue
plt.figure(figsize=(10, 6))
sns.violinplot(
    x="day",         # x-axis: Days of the week (categorical)
    y="total_bill",  # y-axis: Total bill amount (numerical)
    data=tips, 
    hue="sex",       # Color by gender (categorical)
    palette="Set1",  # Color palette
    split=True       # Split violins by hue categories
)

plt.title("Violin Plot with Hue for Total Bill by Day and Gender")
plt.xlabel("Day of the Week")
plt.ylabel("Total Bill ($)")
plt.legend(title="Gender")
plt.show()

In [ ]:
# Scatter Plot with Hue and Size
plt.figure(figsize=(10, 8))
sns.scatterplot(
    x="total_bill", 
    y="tip", 
    data=tips, 
    hue="day",      # Color by day (categorical)
    size="size",    # Vary marker size by size column (numerical)
    sizes=(20, 200),  # Define the size range for markers
    palette="Set1"  # Color palette
)
plt.title("Scatter Plot with Hue and Size for Tips Dataset")
plt.xlabel("Total Bill ($)")
plt.ylabel("Tip ($)")
plt.legend(title="Day")
plt.show()

In [ ]:
# Create a scatterplot using a Relational Plot (relplot)
sns.relplot(x="total_bill", y="tip", data=tips, hue="time", 
            style="sex", size="size", palette="Set1", height=6)
plt.title("Relational Scatter Plot for Tips Dataset")
plt.xlabel("Total Bill ($)")
plt.ylabel("Tip ($)")
plt.savefig('relplot_with_mv.png')
plt.show()

In [ ]:
# Create a facet grid using a Relational Plot (relplot)
g = sns.relplot(x="total_bill", y="tip",
                data=tips, hue="time",
                col="day",  # Separate plots by day (columns)
                row="sex",  # Separate plots by gender (rows)
                palette="Set1",
                height=3,  # Height of each subplot
                aspect=1.2  # Aspect ratio of each subplot
)

# Set titles and labels for the facets
g.set_titles(col_template="{col_name} Day", row_template="{row_name} Gender")
g.set_axis_labels("Total Bill ($)", "Tip ($)")
plt.suptitle("Relational Scatter Plots by Day and Gender")
plt.subplots_adjust(top=0.9)  # Adjust the title position
plt.show()

In [ ]:
# Load the "iris" dataset
iris = sns.load_dataset("iris")

# Pair Plot
sns.set(style="ticks")
sns.pairplot(iris, hue="species", markers=["o", "s", "D"])
plt.show()

In [ ]:
# Load the "iris" dataset
iris = sns.load_dataset("iris")

# Create a Facet Grid of pairwise scatterplots
g = sns.PairGrid(iris, hue="species")
g.map_upper(sns.scatterplot)
g.map_diag(sns.histplot, kde_kws={"color": "k"})
g.map_lower(sns.kdeplot)
g.add_legend()
plt.show()

## matrix plots

In [ ]:
# Import Seaborn
import seaborn as sns

# To see the datasets offered by seaborn
print(sns.get_dataset_names())
# Let's use a dataset offered by seaborn
titanic = sns.load_dataset('titanic')
# To see the top5 rows
titanic.head()

In [ ]:
# Load the Titanic dataset
titanic = sns.load_dataset("titanic")

# Select only numeric columns
# Note: will work assuming that your DataFrame only 
# contains float64 and int64 data types for numeric columns
# If there are other numeric data types in your DataFrame,
# add them to the include list
numeric_cols = titanic.select_dtypes(include=['float64', 'int64'])

# Compute the correlation matrix
correlation_matrix = numeric_cols.corr()

plt.figure(figsize=(10,8))
# Create a heatmap of the correlation matrix
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm")
plt.title("Correlation Heatmap of Titanic Dataset")
plt.show()


Further Reading:
https://seaborn.pydata.org/

Book: Fundamentals of Data Visualization: A Primer on Making Informative and Compelling Figures. Book by Claus O. Wilke. O’Reilly Media.